In [9]:
import random
from http import HTTPStatus
import dashscope
dashscope.api_key = "sk-74375edec5944b90913ec1e9509f797e"


## Tongyi LLM

In [10]:

def call_with_messages(query):
    messages = [{'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': query}]
    response = dashscope.Generation.call(
        model ="qwen-72b-chat",
        messages=messages,
        # set the random seed, optional, default to 1234 if not set
        seed=random.randint(1, 10000),
        
        result_format='message',  # set the result to be "message" format.
    )
    if response.status_code == HTTPStatus.OK:
        return response
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
        return None

In [11]:
response = call_with_messages("Please explain field work, and give me an example?")

In [13]:
response

GenerationResponse(status_code=<HTTPStatus.OK: 200>, request_id='0d21731b-3676-957b-a757-763438cda400', code='', message='', output=GenerationOutput(text=None, choices=[Choice(finish_reason='stop', message=Message({'role': 'assistant', 'content': 'Field work refers to the process of collecting data or conducting research in a natural environment or "in the field," rather than in a laboratory or controlled setting. Field work can be used in a variety of disciplines, including science, social science, and engineering.\nFor example, an archaeologist might conduct field work by excavating a historical site to uncover artifacts and learn more about the people who lived there. A biologist might conduct field work by studying the behavior of animals in their natural habitat. An engineer might conduct field work by testing the structural integrity of a bridge or other piece of infrastructure.\nField work often involves gathering firsthand observations and data through methods such as surveys, 

In [12]:
print(response['output']['choices'][0]['message']['content'])

Field work refers to the process of collecting data or conducting research in a natural environment or "in the field," rather than in a laboratory or controlled setting. Field work can be used in a variety of disciplines, including science, social science, and engineering.
For example, an archaeologist might conduct field work by excavating a historical site to uncover artifacts and learn more about the people who lived there. A biologist might conduct field work by studying the behavior of animals in their natural habitat. An engineer might conduct field work by testing the structural integrity of a bridge or other piece of infrastructure.
Field work often involves gathering firsthand observations and data through methods such as surveys, interviews, or experiments. It can also involve collecting samples, taking measurements, or making detailed notes about the environment or phenomenon being studied.
Overall, field work is an important part of many scientific and academic fields becau

## Tongyi Multimodal

In [85]:
from http import HTTPStatus
import dashscope


def simple_multimodal_conversation_call(image_path):
    """Simple single round multimodal conversation call.
    """
    messages = [
        {
            "role": "user",
            "content": [
                {"image": image_path},
                {"text": "这是什么?"}
            ]
        }
    ]
    response = dashscope.MultiModalConversation.call(model='qwen-vl-plus',
                                                     messages=messages)
    # The response status_code is HTTPStatus.OK indicate success,
    # otherwise indicate request is failed, you can get error code
    # and message from code and message.
    if response.status_code == HTTPStatus.OK:
        print(response)
    else:
        print(response.code)  # The error code.
        print(response.message)  # The error message.


In [86]:
image_path = "https://dashscope.oss-cn-beijing.aliyuncs.com/images/dog_and_girl.jpeg"
#image_path = "/Users/collin/Desktop/llm-app-server-main/apps/dog_and_girl.png"
simple_multimodal_conversation_call(image_path)

{"status_code": 200, "request_id": "683982c9-604e-9c5d-8ca3-1e55dd095855", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": null, "message": {"role": "assistant", "content": [{"text": "图中是一名女子和狗在海滩上，女子笑着看着狗，狗的前爪放在女子的手上，仿佛在握手。"}]}}]}, "usage": {"input_tokens": 1277, "output_tokens": 28}}


## Img Gen

In [91]:
from http import HTTPStatus
from urllib.parse import urlparse, unquote
from pathlib import PurePosixPath
import requests
from dashscope import ImageSynthesis


def simple_call(prompt):
    rsp = ImageSynthesis.call(model=ImageSynthesis.Models.wanx_v1,
                              prompt=prompt,
                              n=4,
                              size='1024*1024')
    if rsp.status_code == HTTPStatus.OK:
        print(rsp.output)
        print(rsp.usage)
        # save file to current directory
        for result in rsp.output.results:
            file_name = PurePosixPath(unquote(urlparse(result.url).path)).parts[-1]
            with open('./wanxiang/%s' % file_name, 'wb+') as f:
                f.write(requests.get(result.url).content)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))



In [92]:

simple_call("Mouse rides elephant")


{"task_id": "6f7b57b4-4664-4921-84f7-17d0c3caf3d7", "task_status": "SUCCEEDED", "results": [{"url": "https://dashscope-result-bj.oss-cn-beijing.aliyuncs.com/1d/5c/20231206/8d820c8d/c4cf331d-b564-4262-b2ca-5f85fac5816f-1.png?Expires=1701937843&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=5rXfWerbYQJ84ycXsg9y8hM90b0%3D"}, {"url": "https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/70/20231206/723609ee/7b9cec57-f632-4b6d-9f0a-9345d97f2723-1.png?Expires=1701937843&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=mld9Ku%2Fa8bzQP7PbODPJJpTY9HE%3D"}, {"url": "https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/06/20231206/723609ee/b4f3c9eb-23a1-4a8e-9621-0eebdf6f2ebd-1.png?Expires=1701937843&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signature=myhdSBcKU4fRUgCF%2F6TnbFoK8AI%3D"}, {"url": "https://dashscope-result-sh.oss-cn-shanghai.aliyuncs.com/1d/eb/20231206/723609ee/027410f0-f676-4df7-ae7b-3b6b3544cbf4-1.png?Expires=1701937843&OSSAccessKeyId=LTAI5tQZd8AEcZX6KZV4G8qL&Signa

### asyncronous 

In [97]:
from http import HTTPStatus

from dashscope import ImageSynthesis


# 创建异步任务
def create_async_task(prompt):
    rsp = ImageSynthesis.async_call(model=ImageSynthesis.Models.wanx_v1,
                                    prompt=prompt,
                                    n=4,
                                    size='1024*1024')
    print(rsp)
    if rsp.status_code == HTTPStatus.OK:
        print("create")
        print(rsp.output)
        print(rsp.usage)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))
    return rsp


# 获取异步任务信息
def fetch_task_status(task):
    status = ImageSynthesis.fetch(task)
    print(status)
    if status.status_code == HTTPStatus.OK:
        print("fetch")
        print(status.output.task_status)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (status.status_code, status.code, status.message))


# 等待异步任务结束
def wait_task(task):
    rsp = ImageSynthesis.wait(task)
    #print(rsp)
    if rsp.status_code == HTTPStatus.OK:
        print("wait")
        print(rsp.output.task_status)
        for result in rsp.output.results:
            file_name = PurePosixPath(unquote(urlparse(result.url).path)).parts[-1]
            with open('./wanxiang/%s' % file_name, 'wb+') as f:
                f.write(requests.get(result.url).content)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))


# 取消异步任务，只有处于PENDING状态的任务才可以取消
def cancel_task(task):
    rsp = ImageSynthesis.cancel(task)
    print(rsp)
    if rsp.status_code == HTTPStatus.OK:
        print(rsp.output.task_status)
    else:
        print('Failed, status_code: %s, code: %s, message: %s' %
              (rsp.status_code, rsp.code, rsp.message))



In [98]:


task_info = create_async_task(prompt='Eagle flying in blue sky')
fetch_task_status(task_info)
wait_task(task_info)


{"status_code": 200, "request_id": "422c28b0-4a8d-96be-b68d-1859f65dbd0b", "code": "", "message": "", "output": {"task_id": "de374a56-3223-48df-8937-a0e2f0beb74b", "task_status": "PENDING", "results": []}, "usage": null}
create
{"task_id": "de374a56-3223-48df-8937-a0e2f0beb74b", "task_status": "PENDING", "results": []}
None
{"status_code": 200, "request_id": "13e9ea64-6897-9a44-abb4-e5da3f7443ad", "code": null, "message": "", "output": {"task_id": "de374a56-3223-48df-8937-a0e2f0beb74b", "task_status": "PENDING", "results": [], "submit_time": "2023-12-06 16:38:26.155", "scheduled_time": "2023-12-06 16:38:26.177", "task_metrics": {"TOTAL": 4, "SUCCEEDED": 0, "FAILED": 0}}, "usage": null}
fetch
PENDING
wait
SUCCEEDED


## Cantonese

In [2]:
import json

with open('./505e602e-9bac-4a12-a00e-6002d0c673a7-1.json') as f:
    data = json.load(f)

In [5]:
data['transcripts'][0]['text']

'我知你好中意车车我球你负责吓，但我都知道你中意发哪利系点样要发拉你咧其实咧有几个言眼嘅，quedara系有啲特家嘅经历嘅。因为一回想当年咧，我几次喺感情嗰头嘅咁咧啊，附近咧就有间叫荔进嘅车啦，就系买啲啤酒块嚟嘅咁咧。当我细细嗰嘅一个小同嘅时候咧，每同经过嗰度咧，就那种震撼嘅感觉，见到啲领车点点令喺阳光底下搬公嘅时候咧，爱上要啲红色嘅东西咧。Ja UM avali ai van tar allque exar que vota AQUíar ser el con。好嘞，叫领林堡经理啦，林堡经理个格比较粗质，就同埋冇乜历史，林布经理几都系上嚟咁可以com al punar el un un volecexigui amb bgoleo即系冇乜好研究啊咁。'

In [ ]:
我知你好中意车车我球你负责吓，但我都知道你中意发哪利系点样要发拉你咧其实咧有几个言眼嘅，quedara系有啲特家嘅经历嘅。
因为一回想当年咧，我几次喺感情嗰头嘅咁咧啊，附近咧就有间叫荔进嘅车啦，就系买啲啤酒块嚟嘅咁咧。
当我细细嗰嘅一个小同嘅时候咧，每同经过嗰度咧，就那种震撼嘅感觉，见到啲领车点点令喺阳光底下搬公嘅时候咧，爱上要啲红色嘅东西咧。
Ja UM avali ai van tar allque exar que vota AQUíar ser el con。好嘞，叫领林堡经理啦，林堡经理个格比较粗质，就同埋冇乜历史，
林布经理几都系上嚟咁可以com al punar el un un volecexigui amb bgoleo即系冇乜好研究啊咁。

In [6]:
with open('./94ccf09d-08c6-4128-9fa5-f67f04a0070c-1.json') as f:
    data = json.load(f)

In [8]:
data['transcripts'][0]['text']

'Síjo sap jo osat有杯奶茶啊，我先食饮，唔杯奶茶喝支烟，哈哈反敷奶茶睇入铺纸料咁嘅。要有时间的话，但系但系系啊啊咁样我都系9点点19点钟咯，食八点半钟食早餐，食到九点钟睇要纸啊，九点钟食食到九点半啊，唔九点半半，但你唔饮奶茶会会痛啊啊冇，我系想点钟有唔会头痛，倒到旁边买一。JARYARR이M이정도양기에고해야니데연착부이야가7杯咗啊七杯嘅奶茶头先又话4英个，而家有一个一般啊，平日你讲咗饮4至5杯嘛，最高嘅饮7杯嘛。但我真得好好犀利噶，饮奶茶，饮奶茶，同饮奶茶都唔同噶，唔知道饮话就会说。第二呢我系弄得就系。'

In [ ]:
Síjo sap jo osat有杯奶茶啊，我先食饮，唔杯奶茶喝支烟，哈哈反敷奶茶睇入铺纸料咁嘅。
要有时间的话，但系但系系啊啊咁样我都系9点点19点钟咯，食八点半钟食早餐，食到九点钟睇要纸啊，
九点钟食食到九点半啊，唔九点半半，但你唔饮奶茶会会痛啊啊冇，我系想点钟有唔会头痛，倒到旁边买一。
JARYARR이M이정도양기에고해야니데연착부이야가7杯咗啊七杯嘅奶茶头先又话4英个，而家有一个一般啊，平日你讲咗饮4至5杯嘛，最高嘅饮7杯嘛。
但我真得好好犀利噶，饮奶茶，饮奶茶，同饮奶茶都唔同噶，唔知道饮话就会说。第二呢我系弄得就系。

# Test

In [1]:
from langchain import hub

In [3]:
!pip install langchainhub

In [7]:
hub.pull('hwchase17/react')

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')